# PyTorch 선형 회귀 구현

In [1]:
import pandas as pd
import numpy as np 
import torch 

import matplotlib.pyplot as plt 
%matplotlib inline

torch.manual_seed(7777)

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['const'] = np.ones(df.shape[0])
df.tail()

C:\Users\gktna\miniconda3\envs\soo\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


# 1.

$X$ 를 Feature, ,$w$를 가중치 벡터, $y$를 Target이라 하자. 

$X^T X$의 역행렬이 존재 한다고 가정했을 때,  

아래의 식을 이용해 $w$의 추정치 $w^*$를 구해봅시다. (PyTorch 연산 사용)

$$
w^{\ast} = (X^TX)^{-1} X^T y
$$

In [4]:
df.shape

(506, 14)

In [6]:
x = torch.tensor(df.values)
y = torch.tensor(boston.target).view(-1, 1)

In [7]:
x.shape, y.shape

(torch.Size([506, 14]), torch.Size([506, 1]))

In [10]:
XT = torch.transpose(x, 0, 1)
XT.shape

torch.Size([14, 506])

In [12]:
w = torch.matmul(torch.mm(torch.linalg.inv(torch.matmul(XT, x)), XT), y)
w

tensor([[-1.0801e-01],
        [ 4.6420e-02],
        [ 2.0559e-02],
        [ 2.6867e+00],
        [-1.7767e+01],
        [ 3.8099e+00],
        [ 6.9222e-04],
        [-1.4756e+00],
        [ 3.0605e-01],
        [-1.2335e-02],
        [-9.5275e-01],
        [ 9.3117e-03],
        [-5.2476e-01],
        [ 3.6459e+01]], dtype=torch.float64)

In [14]:
y_pred = torch.mm(x, w)

print("예측한 집값 :", y_pred[19], "실제 집값 :", boston.target[19])

예측한 집값 : tensor([18.4061], dtype=torch.float64) 실제 집값 : 18.2


---
# 2. Gradient descent 방식

#### w, b 설정 & loss 계산

In [16]:
w = torch.rand((14, 1), dtype=torch.float64, requires_grad=True)
b = torch.rand((1, 1), dtype=torch.float64, requires_grad=True)  
    # const 컬럼을 추가했기 때문에 b가 꼭 필요한건 아님!

In [19]:
z = x.mm(w) + b
loss = torch.mean((z - y) ** 2)

#### 미분값 계산

In [20]:
loss.backward()

In [21]:
w.grad, b.grad

(tensor([[4.6543e+03],
         [1.1592e+04],
         [1.2824e+04],
         [7.4684e+01],
         [6.1311e+02],
         [6.7807e+03],
         [7.6970e+04],
         [3.9565e+03],
         [1.1487e+04],
         [4.6771e+05],
         [2.0181e+04],
         [3.9043e+05],
         [1.4341e+04],
         [1.0850e+03]], dtype=torch.float64),
 tensor([[1084.9567]], dtype=torch.float64))

In [22]:
print(loss)

tensor(303557.1678, dtype=torch.float64, grad_fn=<MeanBackward0>)


- 그냥 loss 값만 출력하고 싶다!
- numpy() 를 사용해보자!

In [23]:
print(loss.numpy())

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

- requires_grad = True 인 경우, numpy() 모듈 사용 불가능.
- requires_grad를 그만두는 torch.no_grad() 나 detach() 중 하나를 사용해야 한다.
- 제일 간단한건  `torch.item()` 모듈 사용

In [24]:
print(loss.detach().numpy())

303557.16782532405


In [25]:
with torch.no_grad():
    print(loss.numpy())

303557.16782532405


In [26]:
loss.item()

303557.16782532405

#### w, b 값 재할당 (assign)
- assign 대신에 data에 접근해서 값을 수정 
- tensor.data = 다른데이터

In [28]:
lr = 0.000003

for epoch in range(100):
    z = x.mm(w) + b
    loss = torch.mean((y - z)**2)
    
    loss.backward()
    
    w.data -= w.grad * lr  # 재할당
    b.data -= b.grad * lr
    
    print("{} - loss : {}".format(epoch, loss.item()))
    
    w.grad.zero_    # 미분값이 누적되는것을 막기 위함
    b.grad.zero_

0 - loss : 303557.16782532405
1 - loss : 2295406.8070335095
2 - loss : 539650.3465805259
3 - loss : 2031799.0480018184
4 - loss : 825037.614489293
5 - loss : 1729338.5293880112
6 - loss : 1142547.9143064006
7 - loss : 1404591.5434815108
8 - loss : 1471707.1735128702
9 - loss : 1075681.4629805093
10 - loss : 1793099.0528657676
11 - loss : 763285.6240031882
12 - loss : 2089380.7184846008
13 - loss : 487146.5766556782
14 - loss : 2343144.693960432
15 - loss : 262557.92599439813
16 - loss : 2537698.0823960975
17 - loss : 101438.17198060958
18 - loss : 2661185.157926967
19 - loss : 14211.317602769817
20 - loss : 2707921.9994103303
21 - loss : 7525.19904706802
22 - loss : 2675675.686541709
23 - loss : 81069.21104010857
24 - loss : 2564791.788789602
25 - loss : 228917.96366084932
26 - loss : 2381109.006384795
27 - loss : 442832.0813088454
28 - loss : 2136975.3824908794
29 - loss : 711779.7100858286
30 - loss : 1847747.9191313416
31 - loss : 1019522.0964519086
32 - loss : 1529184.607560186
33 

#### bakward() 를 안쓰는 방법

In [29]:
lr = 0.000003

for epoch in range(100):
    z = x.mm(w) + b
    loss = torch.mean((y - z)**2)
    
    grads = torch.autograd.grad(loss, [w, b])
    
    w.data -= grads[0] * lr  # 재할당
    b.data -= grads[1] * lr
    
    print("{} - loss : {}".format(epoch, loss.item()))
    
    w.grad.zero_    # 미분값이 누적되는것을 막기 위함
    b.grad.zero_

0 - loss : 65839.53900859304
1 - loss : 50755.865676375266
2 - loss : 39133.245019078175
3 - loss : 30177.237486059206
4 - loss : 23275.82654223587
5 - loss : 17957.511625689407
6 - loss : 13859.032929556117
7 - loss : 10700.513011777482
8 - loss : 8266.309934114777
9 - loss : 6390.268957784455
10 - loss : 4944.361827146665
11 - loss : 3829.9351653279423
12 - loss : 2970.968497807291
13 - loss : 2308.8802319965343
14 - loss : 1798.526033436633
15 - loss : 1405.1157487306527
16 - loss : 1101.8379473885739
17 - loss : 868.0296118091891
18 - loss : 687.7658239530382
19 - loss : 548.7730402891115
20 - loss : 441.59168530294596
21 - loss : 358.93084650911146
22 - loss : 295.1709895477551
23 - loss : 245.98073073900997
24 - loss : 208.02149961341522
25 - loss : 178.719929262709
26 - loss : 156.09243899195513
27 - loss : 138.6100383461557
28 - loss : 125.09412801961099
29 - loss : 114.63618929069429
30 - loss : 106.53588416288858
31 - loss : 100.25334480456043
32 - loss : 95.37239903657719
33

#### optimizer 사용하기

In [30]:
opt = torch.optim.SGD([w, b], lr = lr) # 미분값을 추적한 파라미터를 인수로 입력

for epoch in range(100):    
    z = (x.matmul(w) + b)
    loss = torch.mean((z - y)**2)  
    
    loss.backward()
    opt.step()  # opt 에서 정한 lr 대로 미분값 할당을 작동하라는 의미. 호출만 하면 된다.
    print("{:3} - loss : {}".format(epoch, loss.item()), end="\r")
    
    opt.zero_grad()

In [31]:
y_pred = x.matmul(w) + b
print("예측한 집값 :", y_pred[19].item(), "실제 집값 :", boston.target[19])

예측한 집값 : 24.823975602955297 실제 집값 : 18.2
